In [4]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient("localhost", 27017)
client.drop_database("userData")

db = client.userData
with open("people.json") as f:
    file_data = json.load(f)
db.users.insert_many(file_data)


In [5]:
for x in db.users.aggregate(
    [
        {
            "$project": {
                "_id": 0,
                "location": {
                    "type": "Point",
                    "coordinates": [
                        {
                            "$convert": {
                                "input": "$location.coordinates.longitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                        {
                            "$convert": {
                                "input": "$location.coordinates.latitude",
                                "to": "double",
                                "onError": 0.0,
                                "onNull": 0.0,
                            }
                        },
                    ],
                },
            }
        },
        {"$out": "transformedLocations"},
    ]
):
    pprint(x)


In [6]:
db.transformedLocations.find_one()


{'_id': ObjectId('64570c7db842f828816f52e3'),
 'location': {'type': 'Point', 'coordinates': [-31.0208, -29.8113]}}

In [7]:
client.close()
